import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import ee
import os

In [1]:
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import ee
import os

In [2]:
import ee
import geemap
service_account = ' boba-account@boba-430314.iam.gserviceaccount.com '
credentials = ee.ServiceAccountCredentials(service_account,'C:/Users/arj26323/OneDrive - University of Georgia/Documents/GEE AUTH/boba-430314-a19be859bfca.json')

ee.Initialize(credentials)

print(geemap.__version__)

0.34.1


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


In [3]:
#Variables to add (and then add to Virginia): Landsat min/max over growing season; Daymet May-Oct; Daymet min/max
#Can tidal amplitude and RSLR be worked in?
#Look at autocorrelation

In [4]:
Map = geemap.Map(center=[31.539096,-81.422318], zoom=10)

##Adding every plot coordinate
# allplots_fc = 'C:/Users/arj26323/Documents/Data/Biomass datasets/Sapelo/GA_allplots_NEW.csv'
# fc_all = geemap.csv_to_ee(allplots_fc, latitude = "Latitude", longitude = "Longitude")

allplots_fc = 'C:/Users/arj26323/Documents/Data/Biomass datasets/Sapelo/GA_allplots_NEW.csv'

x = pd.read_csv(allplots_fc).reset_index()
x = x.drop(columns='Flowering_Status')
x = x[x['Species_Code'] != 'ActiveX VT_ERROR: ']
x = x.dropna(subset=['Species_Code'])

fc_all = geemap.pandas_to_ee(x, latitude = "Latitude", longitude = "Longitude")

In [5]:
##Function to cloud mask from the pixel_qa band of Landsat 5/8 SR data.

#IS THIS SAME BETWEEN SENSORS: https://github.com/giswqs/geemap/discussions/672

def maskL8sr(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)
    # Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    # Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(opticalBands, None, True) \
    .addBands(thermalBands, None, True) \
    .updateMask(qaMask) \
    .updateMask(saturationMask)

#NOTE 10/6/2022 - This has been updated for Landsat Collection 2 https://www.usgs.gov/landsat-missions/landsat-collection-2

In [6]:
##TIDAL FILTERING; from Narron et al. 2022
##Utilizes L8 bands 4 and 6 for NDWI, and bands 3 and 4 (for pheno)
##Does it work for Landsat 5?

# def addFLATS(image):
#     flats = ee.Image(0).expression(
#         '1/(1+2.718281828459045**-(-1.57 + 20*(RED-SWIR)/(RED+SWIR) + 68.6*(GREEN-RED)/(GREEN+RED)))', {
#             'SWIR': image.select('SR_B6'),
#             'RED': image.select('SR_B4'),
#             'GREEN': image.select('SR_B3')
#         })
    
#     return image.addBands(flats.rename('flats'))

# def addFLATSL5(image):
#     flats = ee.Image(0).expression(
#         '1/(1+2.718281828459045**-(-1.57 + 20*(RED-SWIR)/(RED+SWIR) + 68.6*(GREEN-RED)/(GREEN+RED)))', {
#             'SWIR': image.select('SR_B5'),
#             'RED': image.select('SR_B3'),
#             'GREEN': image.select('SR_B2')
#         })
    
#     return image.addBands(flats.rename('flats'))

#Updated 10/6/2022 - Band names changed for Collection 2


#12/08/22 - adding CALIBRATED FLATS
def addFLATSL7(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(1.51 + 12.5*(RED-SWIR)/(RED+SWIR) - 41.2*(NIR-RED)/(NIR+6*RED-7.5*BLUE+1)))', {
            'SWIR': image.select('SR_B5'),
            'NIR': image.select('SR_B4'),
            'RED': image.select('SR_B3'),
            'BLUE': image.select('SR_B1')
        })
    
    return image.addBands(flats.rename('flats'))

def addFLATSL5(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(1.51 + 12.5*(0.972*(RED-SWIR)/(RED+SWIR)-0.008) - 41.2*(0.991*(NIR-RED)/(NIR+6*RED-7.5*BLUE+1)-0.0014)))', {
            'SWIR': image.select('SR_B5'),
            'NIR': image.select('SR_B4'),
            'RED': image.select('SR_B3'),
            'BLUE': image.select('SR_B1')
        })
    
    return image.addBands(flats.rename('flats'))


def addFLATSL8(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(1.51 + 12.5*(0.841*(RED-SWIR)/(RED+SWIR) - 0.019) - 41.2*(0.771*(NIR-RED)/(NIR+6*RED-7.5*BLUE+1) + 0.011)))', {
            'SWIR': image.select('SR_B6'),
            'NIR': image.select('SR_B5'),
            'RED': image.select('SR_B4'),
            'BLUE': image.select('SR_B2')
        })
    
    return image.addBands(flats.rename('flats'))

def addFLATSL9(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(1.51 + 12.5*(1.225*(RED-SWIR)/(RED+SWIR) + 0.096) - 41.2*(1.038* (NIR-RED)/(NIR+6*RED-7.5*BLUE+1) - 0.004)))', {
            'SWIR': image.select('SR_B6'),
            'NIR': image.select('SR_B5'),
            'RED': image.select('SR_B4'),
            'BLUE': image.select('SR_B2')
        })
    
    return image.addBands(flats.rename('flats'))

##MASKING FLATS
def maskFLATS(image):
    mask1 = image.select('flats').lte(0.1) #less than or equal to 0.1 - change?
    return image.updateMask(mask1)

##ADDING NDVI (for min/max variables)
def addL5ndvi(image):
    ndvi = image.expression(
        '(NIR-RED)/(RED+NIR)', {
            'NIR': image.select('SR_B4'),
            'RED': image.select('SR_B3'),
            'GREEN': image.select('SR_B2')
        })
    
    return image.addBands(ndvi.rename('ndvi'))

def addL8ndvi(image):
    ndvi = image.expression(
        '(NIR-RED)/(RED+NIR)', {
            'NIR': image.select('SR_B5'),
            'RED': image.select('SR_B4'),
            'GREEN': image.select('SR_B3')
        })
    
    return image.addBands(ndvi.rename('ndvi'))

In [7]:
##Pixel extraction functions - addDate for dateless images/collections
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('imagedate').toInt())

##For Landsat images:
def rasterExtraction(image):
    feature = image.sampleRegions(
        collection = fc_all,
        scale = 30,
        tileScale = 16 #ADDED 10/6/2022 - make sure it doesn't affect results (see thread below)
    )
    return feature

##FOR 10m DEM:
def demExtraction(image):
    feature = image.sampleRegions(
        collection = fc_all,
        scale = 10 
    )
    return feature

##FOR 1m DEM:
def dem1Extraction(image):
    feature = image.sampleRegions(
        collection = fc_all,
        scale = 1, 
        tileScale = 16 #ADDED 11/7/2022

    )
    return feature

#tileScale: https://gis.stackexchange.com/questions/373250/understanding-tilescale-in-earth-engine

In [8]:
##Adding DEM
dem = ee.Image('USGS/3DEP/10m') ##This is 1/3 arc second, or 10 m.
dem1 = ee.ImageCollection('USGS/3DEP/1m')

##Set visualization parameters.
dem_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

Map.addLayer(dem, dem_params, '10m DEM')
# Map.addLayer(dem1, dem_params, '1m DEM')

In [9]:
##Calculating mean pixel values for time periods within each year

##NOTE (11/7/22): got 1 m elevation extraction working, and want to consolidate all variables onto one df

def monthly_Avg (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter and set month
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(5, 10, 'month')) \
                              .mean() \
                              .set({'year': year})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)

##Months and years are lists

years_ls5 = range(2000, 2012)
years_ls7 = range(2012, 2013)
years_ls8 = range(2013, 2022)
years_ls9 = range(2022, 2024)
years_dm = range(2000, 2024)

months = range(5,11)
months_daymet = range(1,12)

ls5_collect = ee.ImageCollection(
    'LANDSAT/LT05/C02/T1_L2'
).filterBounds(fc_all).map(maskL8sr).map(addFLATSL5).map(maskFLATS).map(addL5ndvi)

ls7_collect = ee.ImageCollection(
    'LANDSAT/LE07/C02/T1_L2'
).filterBounds(fc_all).map(maskL8sr).map(addFLATSL7).map(maskFLATS).map(addL5ndvi) ####SHOULD BE addFLATSL5

#Note:Landsat datasets migrated to Collection 2

ls8_collect = ee.ImageCollection(
    'LANDSAT/LC08/C02/T1_L2'
).filterBounds(fc_all).map(maskL8sr).map(addFLATSL8).map(maskFLATS).map(addL8ndvi)

ls9_collect = ee.ImageCollection(
    'LANDSAT/LC09/C02/T1_L2'
).filterBounds(fc_all).map(maskL8sr).map(addFLATSL8).map(maskFLATS).map(addL8ndvi)

monthly_ls5 = monthly_Avg(ls5_collect, years = years_ls5)
monthly_ls7 = monthly_Avg(ls7_collect, years = years_ls7)
monthly_ls8 = monthly_Avg(ls8_collect, years = years_ls8)
monthly_ls9 = monthly_Avg(ls9_collect, years = years_ls9)

monthly_ls5.size().getInfo()

##From stackexchange: calculating monthly averages across many years:
#https://gis.stackexchange.com/questions/290892/google-earth-enginesst-by-month-per-year
#https://gis.stackexchange.com/questions/426662/image-collection-monthly-averages-using-geemap-package

12

In [10]:
Map = geemap.Map(center=[31.539096,-81.422318], zoom=10)

l8_param = {'min': 0, 
             'max': 1, 
             'bands': ['flats'], 
             'gamma': 1}


gax2 = ee.Image('LANDSAT/LC09/C02/T1_L2/LC09_016038_20221022')

Map.addLayer((addFLATSL7(maskL8sr(gax2))), l8_param)
Map.addLayer(maskFLATS(addFLATSL7(maskL8sr(gax2))), l8_param)


gax = ee.Image('LANDSAT/LC08/C02/T1_L2/LC08_017038_20240401')

# Map.addLayer((addFLATSL7(maskL8sr(gax))), l8_param)
# Map.addLayer(maskFLATS(addFLATSL7(maskL8sr(gax))), l8_param)

# Map

In [11]:
def year_Avg (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter and set month
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(1, 12, 'month')) \
                              .mean() \
                              .set({'year': year})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)


year_ls5 = year_Avg(ls5_collect, years = years_ls5)
year_ls7 = year_Avg(ls7_collect, years = years_ls7)
year_ls8 = year_Avg(ls8_collect, years = years_ls8)
year_ls9 = year_Avg(ls9_collect, years = years_ls9)

yearlist_5 = year_ls5.toList(year_ls5.size())
yearlist_7 = year_ls7.toList(year_ls7.size())
yearlist_8 = year_ls8.toList(year_ls8.size())
yearlist_9 = year_ls9.toList(year_ls9.size())

def peak_Avg (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter and set month
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(8, 10, 'month')) \
                              .mean() \
                              .set({'year': year})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)

peak_ls5 = peak_Avg(ls5_collect, years = years_ls5)
peak_ls7 = peak_Avg(ls7_collect, years = years_ls7)
peak_ls8 = peak_Avg(ls8_collect, years = years_ls8)
peak_ls9 = peak_Avg(ls9_collect, years = years_ls9)

peaklist_5 = peak_ls5.toList(peak_ls5.size())
peaklist_7 = peak_ls7.toList(peak_ls7.size())
peaklist_8 = peak_ls8.toList(peak_ls8.size())
peaklist_9 = peak_ls9.toList(peak_ls9.size())

def early_Avg (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter and set month
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(3, 5, 'month')) \
                              .mean() \
                              .set({'year': year})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)

early_ls5 = early_Avg(ls5_collect, years = years_ls5)
early_ls7 = early_Avg(ls7_collect, years = years_ls7)
early_ls8 = early_Avg(ls8_collect, years = years_ls8)
early_ls9 = early_Avg(ls9_collect, years = years_ls9)

earlylist_5 = early_ls5.toList(early_ls5.size())
earlylist_7 = early_ls7.toList(early_ls7.size())
earlylist_8 = early_ls8.toList(early_ls8.size())
earlylist_9 = early_ls9.toList(early_ls9.size())


def MayOct_Avg (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter and set month
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(5, 10, 'month')) \
                              .mean() \
                              .set({'year': year})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)


MayOct_ls5 = MayOct_Avg(ls5_collect, years = years_ls5)
MayOct_ls7 = MayOct_Avg(ls7_collect, years = years_ls7)
MayOct_ls8 = MayOct_Avg(ls8_collect, years = years_ls8)
MayOct_ls9 = MayOct_Avg(ls9_collect, years = years_ls9)

MayOctlist_5 = MayOct_ls5.toList(MayOct_ls5.size())
MayOctlist_7 = MayOct_ls7.toList(MayOct_ls7.size())
MayOctlist_8 = MayOct_ls8.toList(MayOct_ls8.size())
MayOctlist_9 = MayOct_ls9.toList(MayOct_ls9.size())


def MayOct_Max (collection, years):
  maxim = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter and set month
      Monthly_maxim = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(5, 10, 'month')) \
                              .max() \
                              .set({'year': year})
      maxim.append (Monthly_maxim)
  return ee.ImageCollection.fromImages(maxim)

Max_ls5 = MayOct_Max(ls5_collect, years = years_ls5)
Max_ls7 = MayOct_Max(ls7_collect, years = years_ls7)
Max_ls8 = MayOct_Max(ls8_collect, years = years_ls8)
Max_ls9 = MayOct_Max(ls9_collect, years = years_ls9)

Maxlist_5 = Max_ls5.toList(Max_ls5.size())
Maxlist_7 = Max_ls7.toList(Max_ls7.size())
Maxlist_8 = Max_ls8.toList(Max_ls8.size())
Maxlist_9 = Max_ls9.toList(Max_ls9.size())



def MayOct_Min (collection, years):
  minim = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter and set month
      Monthly_minim = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(5, 10, 'month')) \
                              .max() \
                              .set({'year': year})
      minim.append (Monthly_minim)
  return ee.ImageCollection.fromImages(minim)

Min_ls5 = MayOct_Min(ls5_collect, years = years_ls5)
Min_ls7 = MayOct_Min(ls7_collect, years = years_ls7)
Min_ls8 = MayOct_Min(ls8_collect, years = years_ls8)
Min_ls9 = MayOct_Min(ls9_collect, years = years_ls9)

Minlist_5 = Min_ls5.toList(Min_ls5.size())
Minlist_7 = Min_ls7.toList(Min_ls7.size())
Minlist_8 = Min_ls8.toList(Min_ls8.size())
Minlist_9 = Min_ls9.toList(Min_ls9.size())

In [12]:
list_5 = monthly_ls5.toList(monthly_ls5.size())
list_7 = monthly_ls7.toList(monthly_ls7.size())
list_8 = monthly_ls8.toList(monthly_ls8.size())
list_9 = monthly_ls9.toList(monthly_ls9.size())

vis_param = {'min': 0, 
             'max': 0.2, 
             'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 
             'gamma': 1.5}

x= ee.Image(list_5.get(0))
y= ee.Image(list_8.get(0))

Map.addLayer(x, vis_param)

Map.addLayer(fc_all)
# Map

In [13]:
#Daymet
def dm_Avg (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(1, 12, 'month')) \
                              .mean() \
                              .set({'year': year})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)


def dm_earlyseason (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(3, 5, 'month')) \
                              .mean() \
                              .set({'year': year})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)


def peakdm_Avg (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(8, 10, 'month')) \
                              .mean() \
                              .set({'year': year})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)

def rolling_average(daymet_ic):
    # Get the start and end years from the collection
    start_date = ee.Date(daymet_ic.first().get('system:time_start'))
    end_date = ee.Date(daymet_ic.sort('system:time_start', False).first().get('system:time_start'))
    start_year = start_date.get('year').getInfo()
    end_year = end_date.get('year').getInfo()
    
    rolling_avg_ic = ee.ImageCollection([])
    
    # Loop over each year in the collection's range, starting from the third year
    for year in range(start_year + 2, end_year + 1):
        # Define the 3-year range for each year
        start_date = ee.Date.fromYMD(year - 2, 1, 1)
        end_date = ee.Date.fromYMD(year, 12, 31)
        
        # Filter the collection to this 3-year range and calculate mean
        three_year_avg = daymet_ic.filterDate(start_date, end_date).mean()
        
        # Set the year as a property to keep track
        three_year_avg = three_year_avg.set('year', year)
        
        # Add to the rolling average collection
        rolling_avg_ic = rolling_avg_ic.merge(ee.ImageCollection([three_year_avg]))

    return rolling_avg_ic

##COLLECTION
daymet = ee.ImageCollection('NASA/ORNL/DAYMET_V4').filterBounds(fc_all)

#Resampling daymet collection to 30m
def resamp_dm(img):
    resampled_dm = img.reproject(
        crs = 'EPSG:4326',
        scale = 30
    ).resample('bilinear')
    return resampled_dm
resamp_daymet = daymet.map(resamp_dm) #not sure if anything is actually changed

#Over year
monthly_dm = dm_Avg(resamp_daymet, years = years_dm) #Updated to resampled daymet collection!
dm_list = monthly_dm.toList(monthly_dm.size())

#Peak biomass
peak_dm = peakdm_Avg(resamp_daymet, years = years_dm) #Updated to resampled daymet collection!
peakdm_list = peak_dm.toList(peak_dm.size())

#Previous 3 years, rolling average
rolling_av = rolling_average(resamp_daymet) 
rolling_list = rolling_av.toList(rolling_av.size())

#Early season
early_dm = dm_earlyseason(resamp_daymet, years = years_dm) #Updated to resampled daymet collection!
early_list = early_dm.toList(early_dm.size())

#Growing season
MayOct_dm = MayOct_Avg(resamp_daymet, years = years_dm) #Updated to resampled daymet collection!
dm_MayOct_list = MayOct_dm.toList(MayOct_dm.size())

#Max
Max_dm = MayOct_Max(resamp_daymet, years = years_dm) #Updated to resampled daymet collection!
dm_Max_list = Max_dm.toList(Max_dm.size())

#Min
Min_dm = MayOct_Min(resamp_daymet, years = years_dm) #Updated to resampled daymet collection!
dm_Min_list = Min_dm.toList(Min_dm.size())

In [14]:
xxx = daymet.first()
maximum_temperature = xxx.select('tmax')

# Define visualization parameters
maximum_temperature_vis = {
    'min': 11,
    'max': 12,
    'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red']
}

# Create a map
Map = geemap.Map()
Map.addLayer(maximum_temperature, maximum_temperature_vis, 'Maximum Temperature')
# Map

In [15]:
#Gridmet functions, pay attention to date
def gm_Avg (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(1, 12, 'month')) \
                              .mean() \
                              .set({'year': year, 'system:time_start': ee.Date.fromYMD(year, 1, 1).millis()})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)


def gm_earlyseason (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(3, 5, 'month')) \
                              .mean() \
                              .set({'year': year, 'system:time_start': ee.Date.fromYMD(year, 3, 1).millis()})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)


def peakgm_Avg (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(8, 10, 'month')) \
                              .mean() \
                              .set({'year': year, 'system:time_start': ee.Date.fromYMD(year, 8, 1).millis()})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)


def gmMayOct_Avg (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter and set month
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(5, 10, 'month')) \
                              .mean() \
                              .set({'year': year, 'system:time_start': ee.Date.fromYMD(year, 5, 1).millis()})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)

def gmMayOct_Max (collection, years):
  maxim = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter and set month
      Monthly_maxim = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(5, 10, 'month')) \
                              .max() \
                              .set({'year': year, 'system:time_start': ee.Date.fromYMD(year, 5, 1).millis()})
      maxim.append (Monthly_maxim)
  return ee.ImageCollection.fromImages(maxim)

def gmMayOct_Min (collection, years):
  minim = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter and set month
      Monthly_minim = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(5, 10, 'month')) \
                              .max() \
                              .set({'year': year, 'system:time_start': ee.Date.fromYMD(year, 5, 1).millis()})
      minim.append (Monthly_minim)
  return ee.ImageCollection.fromImages(minim)

In [16]:
#FEB 2025 - ADDING GRIDMET DATA (4km resolution I believe)
gridmet = ee.ImageCollection('GRIDMET/DROUGHT').filterBounds(fc_all)

def add_year(image):
    year = ee.Date(image.get("system:time_start")).get("year")
    return image.set("Year", year)

gridmet_resamp = gridmet.map(resamp_dm).map(add_year)

#Over year
monthly_gridmet = gm_Avg(gridmet_resamp, years = years_dm) #Updated to resampled daymet collection!
gridmet_list = monthly_gridmet.toList(monthly_gridmet.size())

#Peak biomass
peak_gridmet = peakgm_Avg(gridmet_resamp, years = years_dm) #Updated to resampled daymet collection!
peakgridmet_list = peak_gridmet.toList(peak_gridmet.size())

#Previous 3 years, rolling average
rolling_gridmet = rolling_average(gridmet_resamp) 
rollinggridmet_list = rolling_gridmet.toList(rolling_gridmet.size())

#Early season
early_gridmet = gm_earlyseason(gridmet_resamp, years = years_dm) #Updated to resampled daymet collection!
earlygridmet_list = early_gridmet.toList(early_gridmet.size())

#Growing season
MayOct_gridmet = gmMayOct_Avg(gridmet_resamp, years = years_dm) #Updated to resampled daymet collection!
gridmet_MayOct_list = MayOct_gridmet.toList(MayOct_gridmet.size())

#Max
Max_gridmet = gmMayOct_Max(gridmet_resamp, years = years_dm) #Updated to resampled daymet collection!
gridmet_Max_list = Max_gridmet.toList(Max_gridmet.size())

#Min
Min_gridmet = gmMayOct_Min(gridmet_resamp, years = years_dm) #Updated to resampled daymet collection!
gridmet_Min_list = Min_gridmet.toList(Min_gridmet.size())

In [17]:
# gridmet.first()

In [18]:
#Elevation
dem_extract = demExtraction(dem)
# dem_vals = geemap.ee_to_csv(dem_extract, 'dem.csv') ##10m dataset
# dem_vals = geemap.ee_to_pandas(dem1.map(dem1Extraction).flatten()) ##ONE METER DATASET

dem_dict = dem_extract.getInfo()

# Extract the properties of the features into a list of dictionaries
features = [feature['properties'] for feature in dem_dict['features']]

# Convert the list of dictionaries into a pandas DataFrame
dem_vals = pd.DataFrame(features)
dem_vals

,Date,Latitude,Longitude,Plant_Biomass,Plot,Quadrat_Area,Sample_Size,Site,Species_Code,Year,Zone,elevation,index
0,2000-10-11,31.538627,-81.422539,516.324,1,0.25,23,1,A1,2000,1,3.035750e-01,0
1,2001-10-17,31.538627,-81.422539,424.416,1,0.25,18,1,A1,2001,1,3.035750e-01,1
2,2002-10-16,31.538627,-81.422539,259.784,1,0.25,16,1,A1,2002,1,3.035750e-01,2
3,2004-10-25,31.538627,-81.422539,436.332,1,0.25,18,1,A1,2004,1,3.035750e-01,3
4,2005-10-24,31.538627,-81.422539,649.300,1,0.25,25,1,A1,2005,1,3.035750e-01,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3893,2022-10-21,31.535648,-81.295100,2132.480,68,0.25,46,2,A1,2022,1,5.200769e-17,3906
3894,2022-10-18,31.538420,-81.422320,350.604,75,0.25,14,1,A1,2022,1,1.175494e-38,3907
3895,2023-10-11,31.538420,-81.422320,615.364,75,0.25,22,1,A1,2023,1,1.175494e-38,3908
3896,2022-10-17,31.307693,-81.415592,1621.272,75,0.25,58,8,A1,2022,1,9.059039e-01,3909


In [18]:
#Yearly:
years_l5 = range(2000, 2012)
years_l7 = range(2012, 2013)
years_l8 = range(2013, 2022)
years_l9 = range(2022, 2024)

# year_extract

landsat5_list = []
for i in range(len(years_l5)):
    ls5_x = rasterExtraction(ee.Image(yearlist_5.get(i)))
    fc_dict = ls5_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample5 = ydf[ydf['Year'] == years_l5[i]]
    landsat5_list.append(sample5) 
    
landsat7_list = []
for i in range(len(years_l7)):
    ls7_x = rasterExtraction(ee.Image(yearlist_7.get(i)))
    fc_dict = ls7_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample7 = ydf[ydf['Year'] == years_l7[i]]
    landsat7_list.append(sample7) 
    
landsat8_list = []
for i in range(len(years_l8)):
    ls8_x = rasterExtraction(ee.Image(yearlist_8.get(i)))
    fc_dict = ls8_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample8 = ydf[ydf['Year'] == years_l8[i]]
    landsat8_list.append(sample8) 

landsat9_list = []
for i in range(len(years_l9)):
    ls9_x = rasterExtraction(ee.Image(yearlist_9.get(i)))
    fc_dict = ls9_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample9 = ydf[ydf['Year'] == years_l9[i]]
    landsat9_list.append(sample9)     
    
l5_extract = pd.concat(landsat5_list)
l7_extract = pd.concat(landsat7_list)
l8_extract = pd.concat(landsat8_list)
l9_extract = pd.concat(landsat9_list)

year_extract = pd.concat([l5_extract,l7_extract,l8_extract,l9_extract])

year_extract


##OLD CODE, BEFORE EE_TO_PANDAS STOPPED WORKING
# landsat5_list = []
# for i in range(len(years_l5)):
#     ls5_x = geemap.ee_to_df(rasterExtraction(ee.Image(yearlist_5.get(i))))
#     sample5 = ls5_x[ls5_x['Year'] == years_l5[i]]
#     landsat5_list.append(sample5) 
    
# landsat7_list = []
# for i in range(len(years_l7)):
#     ls7_x = geemap.ee_to_df(rasterExtraction(ee.Image(yearlist_7.get(i))))
#     sample7 = ls7_x[ls7_x['Year'] == years_l7[i]]
#     landsat7_list.append(sample7) 
    
# landsat8_list = []
# for i in range(len(years_l8)):
#     ls8_x = geemap.ee_to_df(rasterExtraction(ee.Image(yearlist_8.get(i))))
#     sample8 = ls8_x[ls8_x['Year'] == years_l8[i]]
#     landsat8_list.append(sample8) 

# l5_extract = pd.concat(landsat5_list)
# l7_extract = pd.concat(landsat7_list)
# l8_extract = pd.concat(landsat8_list)

# year_extract = pd.concat([l5_extract,l7_extract,l8_extract])

,Date,Latitude,Longitude,Plant_Biomass,Plot,QA_PIXEL,QA_RADSAT,SR_ATMOS_OPACITY,SR_B1,SR_B2,...,Sample_Size,Site,Species_Code,Year,Zone,flats,ndvi,SR_B6,SR_QA_AEROSOL,ST_B10
0,10/11/2000,31.538627,-81.422539,516.324,1,5882.947368,0,254.947368,0.045103,0.055986,...,23,1,A1,2000,1,0.093937,0.292312,NaN,NaN,NaN
19,10/11/2000,31.536199,-81.296118,1152.400,1,5460.480000,0,241.708333,0.048844,0.062316,...,21,2,A1,2000,1,0.130401,0.321849,NaN,NaN,NaN
21,10/18/2000,31.518920,-81.228956,710.396,1,5462.260870,0,247.000000,0.047960,0.064120,...,20,3,A1,2000,1,0.233475,0.285267,NaN,NaN,NaN
33,10/10/2000,31.451676,-81.365498,1534.304,1,5462.400000,0,207.450000,0.040776,0.051302,...,25,4,A1,2000,1,0.407031,0.215222,NaN,NaN,NaN
47,10/10/2000,31.436090,-81.339919,2022.264,1,5440.000000,0,235.952381,0.043530,0.064877,...,25,5,A1,2000,1,0.007976,0.420151,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3173,10/8/2019,31.334597,-81.478039,2365.620,8,21824.000000,0,NaN,0.016119,0.023663,...,14,7,A4,2019,2,0.024921,0.523277,0.089408,154.250000,298.200205
3179,10/8/2019,31.335017,-81.478711,1821.010,12,21824.000000,0,NaN,0.017188,0.024220,...,12,7,A4,2019,1,0.002758,0.583338,0.111881,149.466667,297.772511
3183,10/8/2019,31.334934,-81.478709,2482.584,13,21824.000000,0,NaN,0.017430,0.024653,...,14,7,A4,2019,1,0.010794,0.515524,0.092701,143.466667,297.890774
3187,10/8/2019,31.334682,-81.478564,2511.936,16,21824.000000,0,NaN,0.017430,0.024653,...,17,7,A4,2019,1,0.010794,0.515524,0.092701,143.466667,297.890774


In [19]:
##PEAK EXTRACT

landsat5_list = []
for i in range(len(years_l5)):
    ls5_x = rasterExtraction(ee.Image(peaklist_5.get(i)))
    fc_dict = ls5_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample5 = ydf[ydf['Year'] == years_l5[i]]
    landsat5_list.append(sample5) 
    
landsat7_list = []
for i in range(len(years_l7)):
    ls7_x = rasterExtraction(ee.Image(peaklist_7.get(i)))
    fc_dict = ls7_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample7 = ydf[ydf['Year'] == years_l7[i]]
    landsat7_list.append(sample7) 
    
landsat8_list = []
for i in range(len(years_l8)):
    ls8_x = rasterExtraction(ee.Image(peaklist_8.get(i)))
    fc_dict = ls8_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample8 = ydf[ydf['Year'] == years_l8[i]]
    landsat8_list.append(sample8) 

landsat9_list = []
for i in range(len(years_l9)):
    ls9_x = rasterExtraction(ee.Image(peaklist_9.get(i)))
    fc_dict = ls9_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample9 = ydf[ydf['Year'] == years_l9[i]]
    landsat9_list.append(sample9)     
    
l5_extract = pd.concat(landsat5_list)
l7_extract = pd.concat(landsat7_list)
l8_extract = pd.concat(landsat8_list)
l9_extract = pd.concat(landsat9_list)

peak_extract = pd.concat([l5_extract,l7_extract,l8_extract,l9_extract])

# year_extractlandsat5_list = []
# for i in range(len(years_l5)):
#     ls5_x = geemap.ee_to_pandas(rasterExtraction(ee.Image(peaklist_5.get(i))))
#     sample5 = ls5_x[ls5_x['Year'] == years_l5[i]]
#     landsat5_list.append(sample5) 
    
# landsat7_list = []
# for i in range(len(years_l7)):
#     ls7_x = geemap.ee_to_pandas(rasterExtraction(ee.Image(peaklist_7.get(i))))
#     sample7 = ls7_x[ls7_x['Year'] == years_l7[i]]
#     landsat7_list.append(sample7) 
    
# landsat8_list = []
# for i in range(len(years_l8)):
#     ls8_x = geemap.ee_to_pandas(rasterExtraction(ee.Image(peaklist_8.get(i))))
#     sample8 = ls8_x[ls8_x['Year'] == years_l8[i]]
#     landsat8_list.append(sample8) 

# l5_extract = pd.concat(landsat5_list)
# l7_extract = pd.concat(landsat7_list)
# l8_extract = pd.concat(landsat8_list)

# peak_extract = pd.concat([l5_extract,l7_extract,l8_extract])

peak_extract

,Date,Latitude,Longitude,Plant_Biomass,Plot,QA_PIXEL,QA_RADSAT,SR_ATMOS_OPACITY,SR_B1,SR_B2,...,Sample_Size,Site,Species_Code,Year,Zone,flats,ndvi,SR_B6,SR_QA_AEROSOL,ST_B10
0,10/11/2000,31.538627,-81.422539,516.324,1,5440.000000,0,377.000000,0.036577,0.044519,...,23,1,A1,2000,1,0.107379,0.352109,NaN,NaN,NaN
19,10/18/2000,31.518920,-81.228956,710.396,1,5440.000000,0,274.714286,0.043076,0.060229,...,20,3,A1,2000,1,0.381674,0.308831,NaN,NaN,NaN
31,10/10/2000,31.451676,-81.365498,1534.304,1,5482.666667,0,252.000000,0.032985,0.044017,...,25,4,A1,2000,1,0.683738,0.171927,NaN,NaN,NaN
45,10/10/2000,31.436090,-81.339919,2022.264,1,5440.000000,0,319.714286,0.038071,0.058869,...,25,5,A1,2000,1,0.007745,0.468609,NaN,NaN,NaN
64,10/12/2000,31.388423,-81.279792,1032.632,1,5440.000000,0,380.200000,0.052417,0.069725,...,20,6,A1,2000,1,0.274974,0.286242,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3078,10/8/2019,31.334597,-81.478039,2365.620,8,21824.000000,0,NaN,0.011963,0.019086,...,14,7,A4,2019,2,0.002498,0.651431,0.082439,200.500000,300.123482
3084,10/8/2019,31.335017,-81.478711,1821.010,12,21824.000000,0,NaN,0.013904,0.020055,...,12,7,A4,2019,1,0.000291,0.688841,0.109980,202.666667,299.338192
3088,10/8/2019,31.334934,-81.478709,2482.584,13,21824.000000,0,NaN,0.014234,0.019789,...,14,7,A4,2019,1,0.001975,0.617637,0.083360,202.666667,299.453265
3092,10/8/2019,31.334682,-81.478564,2511.936,16,21824.000000,0,NaN,0.014234,0.019789,...,17,7,A4,2019,1,0.001975,0.617637,0.083360,202.666667,299.453265


In [20]:
##Early EXTRACT

landsat5_list = []
for i in range(len(years_l5)):
    ls5_x = rasterExtraction(ee.Image(earlylist_5.get(i)))
    fc_dict = ls5_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample5 = ydf[ydf['Year'] == years_l5[i]]
    landsat5_list.append(sample5) 
    
landsat7_list = []
for i in range(len(years_l7)):
    ls7_x = rasterExtraction(ee.Image(earlylist_7.get(i)))
    fc_dict = ls7_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample7 = ydf[ydf['Year'] == years_l7[i]]
    landsat7_list.append(sample7) 
    
landsat8_list = []
for i in range(len(years_l8)):
    ls8_x = rasterExtraction(ee.Image(earlylist_8.get(i)))
    fc_dict = ls8_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample8 = ydf[ydf['Year'] == years_l8[i]]
    landsat8_list.append(sample8) 

landsat9_list = []
for i in range(len(years_l9)):
    ls9_x = rasterExtraction(ee.Image(earlylist_9.get(i)))
    fc_dict = ls9_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample9 = ydf[ydf['Year'] == years_l9[i]]
    landsat9_list.append(sample9)     
    
l5_extract = pd.concat(landsat5_list)
l7_extract = pd.concat(landsat7_list)
l8_extract = pd.concat(landsat8_list)
l9_extract = pd.concat(landsat9_list)

early_extract = pd.concat([l5_extract,l7_extract,l8_extract,l9_extract])

In [21]:
##MayOct EXTRACT LANDSAT

#Yearly:
years_l5 = range(2000, 2012)
years_l7 = range(2012, 2013)
years_l8 = range(2013, 2022)
years_l9 = range(2022, 2024)

# MayOct_extract

landsat5_list = []
for i in range(len(years_l5)):
    ls5_x = rasterExtraction(ee.Image(MayOctlist_5.get(i)))
    fc_dict = ls5_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample5 = ydf[ydf['Year'] == years_l5[i]]
    landsat5_list.append(sample5) 
    
landsat7_list = []
for i in range(len(years_l7)):
    ls7_x = rasterExtraction(ee.Image(MayOctlist_7.get(i)))
    fc_dict = ls7_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample7 = ydf[ydf['Year'] == years_l7[i]]
    landsat7_list.append(sample7) 
    
landsat8_list = []
for i in range(len(years_l8)):
    ls8_x = rasterExtraction(ee.Image(MayOctlist_8.get(i)))
    fc_dict = ls8_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample8 = ydf[ydf['Year'] == years_l8[i]]
    landsat8_list.append(sample8) 

landsat9_list = []
for i in range(len(years_l9)):
    ls9_x = rasterExtraction(ee.Image(MayOctlist_9.get(i)))
    fc_dict = ls9_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample9 = ydf[ydf['Year'] == years_l9[i]]
    landsat9_list.append(sample9)     
    
l5_extract = pd.concat(landsat5_list)
l7_extract = pd.concat(landsat7_list)
l8_extract = pd.concat(landsat8_list)
l9_extract = pd.concat(landsat9_list)

MayOct_extract = pd.concat([l5_extract,l7_extract,l8_extract,l9_extract])

MayOct_extract

,Date,Latitude,Longitude,Plant_Biomass,Plot,QA_PIXEL,QA_RADSAT,SR_ATMOS_OPACITY,SR_B1,SR_B2,...,Sample_Size,Site,Species_Code,Year,Zone,flats,ndvi,SR_B6,SR_QA_AEROSOL,ST_B10
0,10/11/2000,31.538627,-81.422539,516.324,1,5440.000000,0,367.300000,0.044076,0.055722,...,23,1,A1,2000,1,0.084796,0.326986,NaN,NaN,NaN
19,10/11/2000,31.536199,-81.296118,1152.400,1,5474.133333,0,304.928571,0.051196,0.067942,...,21,2,A1,2000,1,0.169121,0.338897,NaN,NaN,NaN
21,10/18/2000,31.518920,-81.228956,710.396,1,5474.133333,0,289.933333,0.048560,0.066871,...,20,3,A1,2000,1,0.250408,0.300386,NaN,NaN,NaN
33,10/10/2000,31.451676,-81.365498,1534.304,1,5463.272727,0,238.545455,0.039605,0.052045,...,25,4,A1,2000,1,0.428273,0.234011,NaN,NaN,NaN
47,10/10/2000,31.436090,-81.339919,2022.264,1,5440.000000,0,278.615385,0.043557,0.066665,...,25,5,A1,2000,1,0.006469,0.444670,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3173,10/8/2019,31.334597,-81.478039,2365.620,8,21824.000000,0,NaN,0.012575,0.020196,...,14,7,A4,2019,2,0.023198,0.587283,0.078031,185.333333,302.493024
3179,10/8/2019,31.335017,-81.478711,1821.010,12,21824.000000,0,NaN,0.014710,0.021550,...,12,7,A4,2019,1,0.000853,0.670045,0.110234,180.250000,302.394329
3183,10/8/2019,31.334934,-81.478709,2482.584,13,21824.000000,0,NaN,0.014768,0.021763,...,14,7,A4,2019,1,0.003180,0.615541,0.086199,176.500000,302.487043
3187,10/8/2019,31.334682,-81.478564,2511.936,16,21824.000000,0,NaN,0.014768,0.021763,...,17,7,A4,2019,1,0.003180,0.615541,0.086199,176.500000,302.487043


In [22]:
##Max EXTRACT LANDSAT

landsat5_list = []
for i in range(len(years_l5)):
    ls5_x = rasterExtraction(ee.Image(Maxlist_5.get(i)))
    fc_dict = ls5_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample5 = ydf[ydf['Year'] == years_l5[i]]
    landsat5_list.append(sample5) 
    
landsat7_list = []
for i in range(len(years_l7)):
    ls7_x = rasterExtraction(ee.Image(Maxlist_7.get(i)))
    fc_dict = ls7_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample7 = ydf[ydf['Year'] == years_l7[i]]
    landsat7_list.append(sample7) 
    
landsat8_list = []
for i in range(len(years_l8)):
    ls8_x = rasterExtraction(ee.Image(Maxlist_8.get(i)))
    fc_dict = ls8_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample8 = ydf[ydf['Year'] == years_l8[i]]
    landsat8_list.append(sample8) 

landsat9_list = []
for i in range(len(years_l9)):
    ls9_x = rasterExtraction(ee.Image(Maxlist_9.get(i)))
    fc_dict = ls9_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample9 = ydf[ydf['Year'] == years_l9[i]]
    landsat9_list.append(sample9)     
    
l5_extract = pd.concat(landsat5_list)
l7_extract = pd.concat(landsat7_list)
l8_extract = pd.concat(landsat8_list)
l9_extract = pd.concat(landsat9_list)

Max_extract = pd.concat([l5_extract,l7_extract,l8_extract,l9_extract])

In [23]:
##Min EXTRACT LANDSAT

landsat5_list = []
for i in range(len(years_l5)):
    ls5_x = rasterExtraction(ee.Image(Minlist_5.get(i)))
    fc_dict = ls5_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample5 = ydf[ydf['Year'] == years_l5[i]]
    landsat5_list.append(sample5) 
    
landsat7_list = []
for i in range(len(years_l7)):
    ls7_x = rasterExtraction(ee.Image(Minlist_7.get(i)))
    fc_dict = ls7_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample7 = ydf[ydf['Year'] == years_l7[i]]
    landsat7_list.append(sample7) 
    
landsat8_list = []
for i in range(len(years_l8)):
    ls8_x = rasterExtraction(ee.Image(Minlist_8.get(i)))
    fc_dict = ls8_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample8 = ydf[ydf['Year'] == years_l8[i]]
    landsat8_list.append(sample8) 

landsat9_list = []
for i in range(len(years_l9)):
    ls9_x = rasterExtraction(ee.Image(Minlist_9.get(i)))
    fc_dict = ls9_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample9 = ydf[ydf['Year'] == years_l9[i]]
    landsat9_list.append(sample9)     
    
l5_extract = pd.concat(landsat5_list)
l7_extract = pd.concat(landsat7_list)
l8_extract = pd.concat(landsat8_list)
l9_extract = pd.concat(landsat9_list)

Min_extract = pd.concat([l5_extract,l7_extract,l8_extract,l9_extract])

In [19]:
##Daymet for loop and elevation
# years_dm = range(2000, 2023)

daymet_list = []
for i in range(len(years_dm)):
    dm_x = rasterExtraction(ee.Image(dm_list.get(i)))
    fc_dict = dm_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sampledm = ydf[ydf['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 

# daymet_list = []
# for i in range(len(years_dm)):
#     dm_x = geemap.ee_to_pandas(rasterExtraction(ee.Image(dm_list.get(i))))
#     sampledm = dm_x[dm_x['Year'] == years_dm[i]]
#     daymet_list.append(sampledm) 
    
daymet_extract = pd.concat(daymet_list)

daymet_extract

,Date,Latitude,Longitude,Plant_Biomass,Plot,Quadrat_Area,Sample_Size,Site,Species_Code,Year,Zone,dayl,index,prcp,srad,swe,tmax,tmin,vp
0,2000-10-11,31.538627,-81.422539,516.324,1,0.25,23,1,A1,2000,1,43200.085938,0,2.489973,343.573212,0,25.069397,14.079397,1765.701416
23,2000-10-18,31.518920,-81.228956,710.396,1,0.25,20,3,A1,2000,1,43200.085938,25,2.319699,336.004730,0,24.539864,15.079507,1874.970703
35,2000-10-10,31.451676,-81.365498,1534.304,1,0.25,25,4,A1,2000,1,43200.085938,37,2.472329,340.215027,0,24.868795,14.635425,1827.156250
49,2000-10-10,31.436090,-81.339919,2022.264,1,0.25,25,5,A1,2000,1,43200.085938,51,2.445726,340.010864,0,24.818274,14.757424,1839.845947
72,2000-10-12,31.388423,-81.279792,1032.632,1,0.25,20,6,A1,2000,1,43200.085938,74,2.393671,334.624725,0,24.615616,15.242548,1893.765625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3511,2023-10-10,31.351949,-81.333980,2508.364,63,0.25,49,9,A1,2023,1,43200.085938,3889,3.621973,331.606934,0,26.301891,15.668713,1918.989624
3517,2023-10-04,31.388000,-81.279858,527.736,64,0.25,17,6,A1,2023,1,43200.085938,3895,3.923589,330.454163,0,26.181644,15.778575,1932.754517
3525,2023-10-04,31.387776,-81.279719,869.096,66,0.25,22,6,A1,2023,1,43200.085938,3903,3.923589,330.454163,0,26.181644,15.778575,1932.754517
3527,2023-10-11,31.538420,-81.422320,615.364,75,0.25,22,1,A1,2023,1,43200.085938,3908,3.915781,330.411926,0,26.379999,15.117753,1860.759888


In [20]:
#Daymet peak time
years_dm = range(2000, 2023)

daymet_list = []
for i in range(len(years_dm)):
    dm_x = rasterExtraction(ee.Image(peakdm_list.get(i)))
    fc_dict = dm_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sampledm = ydf[ydf['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
# daymet_list = []
# for i in range(len(years_dm)):
#     dm_x = geemap.ee_to_pandas(rasterExtraction(ee.Image(peakdm_list.get(i))))
#     sampledm = dm_x[dm_x['Year'] == years_dm[i]]
#     daymet_list.append(sampledm) 
    
peakdaymet_extract = pd.concat(daymet_list)

peakdaymet_extract

,Date,Latitude,Longitude,Plant_Biomass,Plot,Quadrat_Area,Sample_Size,Site,Species_Code,Year,Zone,dayl,index,prcp,srad,swe,tmax,tmin,vp
0,2000-10-11,31.538627,-81.422539,516.324,1,0.25,23,1,A1,2000,1,43642.554688,0,3.261631,346.619019,0,28.971413,18.999348,2280.965332
23,2000-10-18,31.518920,-81.228956,710.396,1,0.25,20,3,A1,2000,1,43642.554688,25,2.840109,337.377716,0,28.514999,19.984131,2415.145996
35,2000-10-10,31.451676,-81.365498,1534.304,1,0.25,25,4,A1,2000,1,43640.546875,37,3.208369,342.130432,0,28.807716,19.541521,2353.217041
49,2000-10-10,31.436090,-81.339919,2022.264,1,0.25,25,5,A1,2000,1,43640.546875,51,3.210978,341.652161,0,28.765326,19.660870,2369.350830
72,2000-10-12,31.388423,-81.279792,1032.632,1,0.25,20,6,A1,2000,1,43639.546875,74,3.313043,333.892059,0,28.587391,20.133043,2434.780029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3507,2022-10-17,31.307529,-81.415324,737.372,58,0.25,8,8,A2,2022,1,43747.886719,3879,5.132609,339.826080,0,29.388586,18.985762,2289.003418
3510,2022-10-20,31.351949,-81.333980,1245.160,63,0.25,27,9,A1,2022,1,43749.136719,3888,5.745435,337.808685,0,29.218803,19.154348,2311.670166
3516,2022-10-15,31.388000,-81.279858,819.116,64,0.25,16,6,A1,2022,1,43749.136719,3894,6.227826,337.447296,0,29.062500,19.304892,2331.447754
3526,2022-10-18,31.538420,-81.422320,350.604,75,0.25,14,1,A1,2022,1,43752.890625,3907,4.347609,348.728699,0,29.400652,18.574131,2237.086670


In [21]:
#Rolling daymet extraction
years_dm = range(2000, 2023)

daymet_list = []
for i in range(len(years_dm)):
    dm_x = rasterExtraction(ee.Image(rolling_list.get(i)))
    fc_dict = dm_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sampledm = ydf[ydf['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
rolling_extract = pd.concat(daymet_list)

rolling_extract

EEException: User memory limit exceeded.

In [22]:
#MayOct daymet extraction
years_dm = range(2000, 2023)

daymet_list = []
for i in range(len(years_dm)):
    dm_x = rasterExtraction(ee.Image(dm_MayOct_list.get(i)))
    fc_dict = dm_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sampledm = ydf[ydf['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
dm_MayOct_extract = pd.concat(daymet_list)

dm_MayOct_extract

,Date,Latitude,Longitude,Plant_Biomass,Plot,Quadrat_Area,Sample_Size,Site,Species_Code,Year,Zone,dayl,index,prcp,srad,swe,tmax,tmin,vp
0,2000-10-11,31.538627,-81.422539,516.324,1,0.25,23,1,A1,2000,1,46699.050781,0,2.587174,370.686676,0,30.488533,20.132500,2385.324951
23,2000-10-18,31.518920,-81.228956,710.396,1,0.25,20,3,A1,2000,1,46699.050781,25,2.389837,361.504395,0,29.986740,21.127010,2522.228516
35,2000-10-10,31.451676,-81.365498,1534.304,1,0.25,25,4,A1,2000,1,46683.062500,37,2.603098,366.274567,0,30.238533,20.647446,2461.446045
49,2000-10-10,31.436090,-81.339919,2022.264,1,0.25,25,5,A1,2000,1,46683.062500,51,2.586304,365.446747,0,30.179184,20.760597,2476.378906
72,2000-10-12,31.388423,-81.279792,1032.632,1,0.25,20,6,A1,2000,1,46675.089844,74,2.585706,357.926727,0,29.934185,21.215652,2542.013184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3507,2022-10-17,31.307529,-81.415324,737.372,58,0.25,8,8,A2,2022,1,46716.511719,3879,4.529783,357.960327,0,30.720869,20.395163,2462.280029
3510,2022-10-20,31.351949,-81.333980,1245.160,63,0.25,27,9,A1,2022,1,46724.585938,3888,4.815761,357.572449,0,30.562880,20.569130,2479.649902
3516,2022-10-15,31.388000,-81.279858,819.116,64,0.25,16,6,A1,2022,1,46724.585938,3894,5.247500,356.140717,0,30.418478,20.720381,2500.445801
3526,2022-10-18,31.538420,-81.422320,350.604,75,0.25,14,1,A1,2022,1,46748.886719,3907,4.351739,359.242950,0,30.804783,20.081739,2416.799316


In [23]:
#Max daymet extraction
years_dm = range(2000, 2023)

daymet_list = []
for i in range(len(years_dm)):
    dm_x = rasterExtraction(ee.Image(dm_Max_list.get(i)))
    fc_dict = dm_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sampledm = ydf[ydf['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
dm_Max_extract = pd.concat(daymet_list)

dm_Max_extract

,Date,Latitude,Longitude,Plant_Biomass,Plot,Quadrat_Area,Sample_Size,Site,Species_Code,Year,Zone,dayl,index,prcp,srad,swe,tmax,tmin,vp
0,2000-10-11,31.538627,-81.422539,516.324,1,0.25,23,1,A1,2000,1,50610.128906,0,29.420000,513.119995,0,39.959999,25.549999,3238.219971
23,2000-10-18,31.518920,-81.228956,710.396,1,0.25,20,3,A1,2000,1,50610.128906,25,30.010000,507.760010,0,39.759998,26.400000,3417.370117
35,2000-10-10,31.451676,-81.365498,1534.304,1,0.25,25,4,A1,2000,1,50576.109375,37,31.879999,509.329987,0,39.840000,25.990000,3335.010010
49,2000-10-10,31.436090,-81.339919,2022.264,1,0.25,25,5,A1,2000,1,50576.109375,51,31.809999,508.769989,0,39.849998,26.090000,3352.919922
72,2000-10-12,31.388423,-81.279792,1032.632,1,0.25,20,6,A1,2000,1,50559.128906,74,32.410000,504.790009,0,39.900002,26.469999,3429.219971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3507,2022-10-17,31.307529,-81.415324,737.372,58,0.25,8,8,A2,2022,1,50542.179688,3879,51.480000,536.000000,0,38.990002,25.230000,3210.600098
3510,2022-10-20,31.351949,-81.333980,1245.160,63,0.25,27,9,A1,2022,1,50559.128906,3888,59.860001,537.320007,0,39.040001,25.520000,3265.600098
3516,2022-10-15,31.388000,-81.279858,819.116,64,0.25,16,6,A1,2022,1,50559.128906,3894,68.809998,538.700012,0,39.080002,25.770000,3314.830078
3526,2022-10-18,31.538420,-81.422320,350.604,75,0.25,14,1,A1,2022,1,50610.128906,3907,64.260002,538.280029,0,38.970001,25.020000,3171.030029


In [24]:
#Min daymet extraction
years_dm = range(2000, 2023)

daymet_list = []
for i in range(len(years_dm)):
    dm_x = rasterExtraction(ee.Image(dm_Min_list.get(i)))
    fc_dict = dm_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sampledm = ydf[ydf['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
dm_Min_extract = pd.concat(daymet_list)

dm_Min_extract

,Date,Latitude,Longitude,Plant_Biomass,Plot,Quadrat_Area,Sample_Size,Site,Species_Code,Year,Zone,dayl,index,prcp,srad,swe,tmax,tmin,vp
0,2000-10-11,31.538627,-81.422539,516.324,1,0.25,23,1,A1,2000,1,50610.128906,0,29.420000,513.119995,0,39.959999,25.549999,3238.219971
23,2000-10-18,31.518920,-81.228956,710.396,1,0.25,20,3,A1,2000,1,50610.128906,25,30.010000,507.760010,0,39.759998,26.400000,3417.370117
35,2000-10-10,31.451676,-81.365498,1534.304,1,0.25,25,4,A1,2000,1,50576.109375,37,31.879999,509.329987,0,39.840000,25.990000,3335.010010
49,2000-10-10,31.436090,-81.339919,2022.264,1,0.25,25,5,A1,2000,1,50576.109375,51,31.809999,508.769989,0,39.849998,26.090000,3352.919922
72,2000-10-12,31.388423,-81.279792,1032.632,1,0.25,20,6,A1,2000,1,50559.128906,74,32.410000,504.790009,0,39.900002,26.469999,3429.219971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3507,2022-10-17,31.307529,-81.415324,737.372,58,0.25,8,8,A2,2022,1,50542.179688,3879,51.480000,536.000000,0,38.990002,25.230000,3210.600098
3510,2022-10-20,31.351949,-81.333980,1245.160,63,0.25,27,9,A1,2022,1,50559.128906,3888,59.860001,537.320007,0,39.040001,25.520000,3265.600098
3516,2022-10-15,31.388000,-81.279858,819.116,64,0.25,16,6,A1,2022,1,50559.128906,3894,68.809998,538.700012,0,39.080002,25.770000,3314.830078
3526,2022-10-18,31.538420,-81.422320,350.604,75,0.25,14,1,A1,2022,1,50610.128906,3907,64.260002,538.280029,0,38.970001,25.020000,3171.030029


In [25]:
#Early daymet extraction
years_dm = range(2000, 2023)

daymet_list = []
for i in range(len(years_dm)):
    dm_x = rasterExtraction(ee.Image(early_list.get(i)))
    fc_dict = dm_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sampledm = ydf[ydf['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
earlydaymet_extract = pd.concat(daymet_list)

earlydaymet_extract

,Date,Latitude,Longitude,Plant_Biomass,Plot,Quadrat_Area,Sample_Size,Site,Species_Code,Year,Zone,dayl,index,prcp,srad,swe,tmax,tmin,vp
0,2000-10-11,31.538627,-81.422539,516.324,1,0.25,23,1,A1,2000,1,45943.253906,0,2.505543,396.020203,0,25.948805,14.393587,1704.089600
23,2000-10-18,31.518920,-81.228956,710.396,1,0.25,20,3,A1,2000,1,45943.253906,25,2.483478,388.017059,0,25.276522,15.525543,1824.977295
35,2000-10-10,31.451676,-81.365498,1534.304,1,0.25,25,4,A1,2000,1,45930.796875,37,2.488369,391.955231,0,25.655544,14.990217,1766.957764
49,2000-10-10,31.436090,-81.339919,2022.264,1,0.25,25,5,A1,2000,1,45930.796875,51,2.501739,392.073914,0,25.583370,15.120326,1781.306396
72,2000-10-12,31.388423,-81.279792,1032.632,1,0.25,20,6,A1,2000,1,45924.582031,74,2.565000,386.231750,0,25.298044,15.637609,1839.727051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3507,2022-10-17,31.307529,-81.415324,737.372,58,0.25,8,8,A2,2022,1,45817.472656,3879,3.188152,385.433685,0,26.585760,14.875109,1766.557739
3510,2022-10-20,31.351949,-81.333980,1245.160,63,0.25,27,9,A1,2022,1,45823.449219,3888,3.176196,384.493164,0,26.409130,15.089022,1786.253540
3516,2022-10-15,31.388000,-81.279858,819.116,64,0.25,16,6,A1,2022,1,45823.449219,3894,3.215326,382.284241,0,26.248369,15.268261,1810.892944
3526,2022-10-18,31.538420,-81.422320,350.604,75,0.25,14,1,A1,2022,1,45841.425781,3907,2.679456,387.008270,0,26.675434,14.459348,1711.481323


In [26]:
# cx = add_year(gridmet.first())
# cx.get('Year')

# Map.addLayer(cx)
# Map

# vc = (ee.Image(gridmet_list.get(0)))


In [27]:
#FEB 2025 - adding PDSI and Potential Evap Transp

#Gridmet season
years_dm = range(2000, 2023)

def add_year(image):
    return image.set("Year", ee.Date(image.date()).get("year"))

daymet_list = []
for i in range(len(years_dm)):
    dm_x = rasterExtraction(add_year(ee.Image(gridmet_list.get(i))))
    fc_dict = dm_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sampledm = ydf[ydf['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
gridmet_extract = pd.concat(daymet_list)


#Gridmet peak time
years_dm = range(2000, 2023)

daymet_list = []
for i in range(len(years_dm)):
    dm_x = rasterExtraction(add_year(ee.Image(peakgridmet_list.get(i))))
    fc_dict = dm_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sampledm = ydf[ydf['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
peakgridmet_extract = pd.concat(daymet_list)

# #Rolling gridmet extraction
# years_dm = range(2000, 2023)

# daymet_list = []
# for i in range(len(years_dm)):
#     dm_x = rasterExtraction(add_year(ee.Image(rollinggridmet_list.get(i))))
#     fc_dict = dm_x.getInfo()
#     features = [feature['properties'] for feature in fc_dict['features']]
#     ydf = pd.DataFrame(features)
#     sampledm = ydf[ydf['Year'] == years_dm[i]]
#     daymet_list.append(sampledm) 
    
# rollinggridmet_extract = pd.concat(daymet_list)

#MayOct gridmet extraction
years_dm = range(2000, 2023)

daymet_list = []
for i in range(len(years_dm)):
    dm_x = rasterExtraction(add_year(ee.Image(gridmet_MayOct_list.get(i))))
    fc_dict = dm_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sampledm = ydf[ydf['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
gridmet_MayOct_extract = pd.concat(daymet_list)

#Max gridmet extraction
years_dm = range(2000, 2023)

daymet_list = []
for i in range(len(years_dm)):
    dm_x = rasterExtraction(add_year(ee.Image(gridmet_Max_list.get(i))))
    fc_dict = dm_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sampledm = ydf[ydf['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
gridmet_Max_extract = pd.concat(daymet_list)

#Min gridmet extraction
years_dm = range(2000, 2023)

daymet_list = []
for i in range(len(years_dm)):
    dm_x = rasterExtraction(add_year(ee.Image(gridmet_Min_list.get(i))))
    fc_dict = dm_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sampledm = ydf[ydf['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
gridmet_Min_extract = pd.concat(daymet_list)

#Early gridmet extraction
years_dm = range(2000, 2023)

daymet_list = []
for i in range(len(years_dm)):
    dm_x = rasterExtraction(add_year(ee.Image(earlygridmet_list.get(i))))
    fc_dict = dm_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sampledm = ydf[ydf['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
earlygridmet_extract = pd.concat(daymet_list)


In [43]:
#Final steps - extra rows in merge?

# landsat_extract = MayOct_extract ##NOT sure if this is the right way to do this - 
#where are all these dfs stored? (in streamlined folder in biomass datasets/sapelo)

#rolling_extract, earlydaymet_extract, early_extract (Landsat)

#MayOct and elevation
landsat_extract = MayOct_extract
dfx = pd.merge(landsat_extract, dem_vals, how = 'left')

#dfx and AugOct
peak_extract.rename(
    columns={
        'SR_B1':'SR_B1_peak','SR_B2':'SR_B2_peak','SR_B3':'SR_B3_peak','SR_B4':'SR_B4_peak', 'SR_B5':'SR_B5_peak',
        'SR_B6':'SR_B6_peak', 'SR_B7':'SR_B7_peak', 'flats':'flats_peak'
    }, inplace=True
)

dfx1 = pd.merge(dfx, peak_extract, on = ['Plant_Biomass', 'Plot', 'Zone', 'Site', 'Year'], how='outer',
                suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

#dfx1 and year_extract

year_extract.rename(
    columns={
        'SR_B1':'SR_B1_year','SR_B2':'SR_B2_year','SR_B3':'SR_B3_year','SR_B4':'SR_B4_year', 'SR_B5':'SR_B5_year',
        'SR_B6':'SR_B6_year', 'SR_B7':'SR_B7_year', 'flats':'flats_year'
    }, inplace=True
)

dfx2 = pd.merge(dfx1, year_extract, on = ['Plant_Biomass', 'Plot', 'Zone', 'Site', 'Year'], how='outer',
                suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')



#dfx2 and early_extract
early_extract.rename(
    columns={
        'SR_B1':'SR_B1_MarMay','SR_B2':'SR_B2_MarMay','SR_B3':'SR_B3_MarMay','SR_B4':'SR_B4_MarMay', 'SR_B5':'SR_B5_MarMay',
        'SR_B6':'SR_B6_MarMay', 'SR_B7':'SR_B7_MarMay', 'flats':'flats_MarMay'
    }, inplace=True
)

dfx2_a = pd.merge(dfx2, early_extract, on = ['Plant_Biomass', 'Plot', 'Zone', 'Site', 'Year'], how='outer',
                suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

#dfx2_a and max
Max_extract.rename(
    columns={
        'SR_B1':'SR_B1_Max','SR_B2':'SR_B2_Max','SR_B3':'SR_B3_Max','SR_B4':'SR_B4_Max', 'SR_B5':'SR_B5_Max',
        'SR_B6':'SR_B6_Max', 'SR_B7':'SR_B7_Max', 'flats':'flats_Max'
    }, inplace=True
)

dfx2_b = pd.merge(dfx2_a, Max_extract, on = ['Plant_Biomass', 'Plot', 'Zone', 'Site', 'Year'], how='outer',
                suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

#dfx2_b and min
Min_extract.rename(
    columns={
        'SR_B1':'SR_B1_Min','SR_B2':'SR_B2_Min','SR_B3':'SR_B3_Min','SR_B4':'SR_B4_Min', 'SR_B5':'SR_B5_Min',
        'SR_B6':'SR_B6_Min', 'SR_B7':'SR_B7_Min', 'flats':'flats_Min'
    }, inplace=True
)

dfx2_c = pd.merge(dfx2_b, Min_extract, on = ['Plant_Biomass', 'Plot', 'Zone', 'Site', 'Year'], how='outer',
                suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')


#dfx2 and daymet mayoct OR YEAR
dfx3 = pd.merge(dfx2_c, daymet_extract, on = ['Plant_Biomass', 'Plot', 'Zone', 'Site', 'Year'], how='right')

#dfx3 and daymet rolling average
rolling_extract.rename(
    columns={
        'swe':'swe_rolling','tmax':'tmax_rolling','tmin':'tmin_rolling','srad':'srad_rolling', 'vp':'vp_rolling',
        'prcp':'prcp_rolling', 'dayl':'dayl_rolling'
    }, inplace=True
)

dfx3_a = pd.merge(dfx3, rolling_extract, on = ['Plant_Biomass', 'Plot', 'Zone', 'Site', 'Year'], how='right',
              suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

#dfx3_a and daymet early (marMay)
earlydaymet_extract.rename(
    columns={
        'swe':'swe_daymMarMay','tmax':'tmax_daymMarMay','tmin':'tmin_daymMarMay','srad':'srad_daymMarMay', 'vp':'vp_daymMarMay',
        'prcp':'prcp_daymMarMay', 'dayl':'dayl_daymMarMay'
    }, inplace=True
)

dfx3_b = pd.merge(dfx3_a, earlydaymet_extract, on = ['Plant_Biomass', 'Plot', 'Zone', 'Site', 'Year'], how='right',
              suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

#dfx3_b and daymet growing (MayOct)
dm_MayOct_extract.rename(
    columns={
        'swe':'swe_daymMayOct','tmax':'tmax_daymMayOct','tmin':'tmin_daymMayOct','srad':'srad_daymMayOct', 'vp':'vp_daymMayOct',
        'prcp':'prcp_daymMayOct', 'dayl':'dayl_daymMayOct'
    }, inplace=True
)

dfx3_c = pd.merge(dfx3_b, dm_MayOct_extract, on = ['Plant_Biomass', 'Plot', 'Zone', 'Site', 'Year'], how='right',
              suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')


#dfx3_c and daymet max (MayOct)
dm_Max_extract.rename(
    columns={
        'swe':'swe_Max','tmax':'tmax_Max','tmin':'tmin_Max','srad':'srad_Max', 'vp':'vp_Max',
        'prcp':'prcp_Max', 'dayl':'dayl_Max'
    }, inplace=True
)

dfx3_d = pd.merge(dfx3_c, dm_Max_extract, on = ['Plant_Biomass', 'Plot', 'Zone', 'Site', 'Year'], how='right',
              suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')


#dfx3_d and daymet min (MayOct)
dm_Min_extract.rename(
    columns={
        'swe':'swe_Min','tmax':'tmax_Min','tmin':'tmin_Min','srad':'srad_Min', 'vp':'vp_Min',
        'prcp':'prcp_Min', 'dayl':'dayl_Min'
    }, inplace=True
)

dfx3_e = pd.merge(dfx3_d, dm_Min_extract, on = ['Plant_Biomass', 'Plot', 'Zone', 'Site', 'Year'], how='right',
              suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

#dfx3_e and daymet peak
peakdaymet_extract.rename(
    columns={
        'swe':'swe_peak','tmax':'tmax_peak','tmin':'tmin_peak','srad':'srad_peak', 'vp':'vp_peak',
        'prcp':'prcp_peak', 'dayl':'dayl_peak'
    }, inplace=True
)

dfx3_f = pd.merge(dfx3_e, peakdaymet_extract, on = ['Plant_Biomass', 'Plot', 'Zone', 'Site', 'Year'], how='right',
              suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')


#GRIDMET ADDED
#dfx3_f and gridmet mayoct OR YEAR
dfx4_a = pd.merge(dfx3_f, gridmet_extract, on = ['Plant_Biomass', 'Plot', 'Zone', 'Site', 'Year'], how='right',
                 suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

#REMOVED - note the jump from dfx4_a to dfx4_c!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# #dfx4_a and gridmet rolling average
# rollinggridmet_extract.rename(    
#     columns={
#         'spei30d':'spei30d_rolling','spei90d':'spei90d_rolling','spei180d':'spei180d_rolling','spei1y':'spei1y_rolling', 
#         'spei5y':'spei5y_rolling',
#         'pdsi':'pdsi_rolling', 'pz':'pz_rolling'
#     }, inplace=True
# )

# dfx4_b = pd.merge(dfx4_a, rollinggridmet_extract, on = ['Plant_Biomass', 'Plot', 'Zone', 'Site', 'Year'], how='right',
#               suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')


#dfx4_b and gridmet early (marMay)
earlygridmet_extract.rename(
    columns={
        'spei30d':'spei30d_MarMay','spei90d':'spei90d_MarMay','spei180d':'spei180d_MarMay','spei1y':'spei1y_MarMay', 
        'spei5y':'spei5y_MarMay',
        'pdsi':'pdsi_MarMay', 'pz':'pz_MarMay'
    }, inplace=True
)

dfx4_c = pd.merge(dfx4_a, earlygridmet_extract, on = ['Plant_Biomass', 'Plot', 'Zone', 'Site', 'Year'], how='right',
              suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

#dfx4_c and gridmet growing (MayOct)
gridmet_MayOct_extract.rename(
    columns={
        'spei30d':'spei30d_MayOct','spei90d':'spei90d_MayOct','spei180d':'spei180d_MayOct','spei1y':'spei1y_MayOct', 
        'spei5y':'spei5y_MayOct',
        'pdsi':'pdsi_MayOct', 'pz':'pz_MayOct'
    }, inplace=True
)

dfx4_d = pd.merge(dfx4_c, gridmet_MayOct_extract, on = ['Plant_Biomass', 'Plot', 'Zone', 'Site', 'Year'], how='right',
              suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')


#dfx4_d and gridmet max (MayOct)
gridmet_Max_extract.rename(
    columns={
        'spei30d':'spei30d_Max','spei90d':'spei90d_Max','spei180d':'spei180d_Max','spei1y':'spei1y_Max', 
        'spei5y':'spei5y_Max',
        'pdsi':'pdsi_Max', 'pz':'pz_Max'
    }, inplace=True
)

dfx4_e = pd.merge(dfx4_d, gridmet_Max_extract, on = ['Plant_Biomass', 'Plot', 'Zone', 'Site', 'Year'], how='right',
              suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')


#dfx4_e and gridmet min (MayOct)
gridmet_Min_extract.rename(
    columns={
        'spei30d':'spei30d_Min','spei90d':'spei90d_Min','spei180d':'spei180d_Min','spei1y':'spei1y_Min', 
        'spei5y':'spei5y_Min',
        'pdsi':'pdsi_Min', 'pz':'pz_Min'
    }, inplace=True
)

dfx4_f = pd.merge(dfx4_e, gridmet_Min_extract, on = ['Plant_Biomass', 'Plot', 'Zone', 'Site', 'Year'], how='right',
              suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

#dfx4_f and gridmet peak
peakgridmet_extract.rename(
    columns={
        'spei30d':'spei30d_gridmet','spei90d':'spei90d_gridmet','spei180d':'spei180d_gridmet','spei1y':'spei1y_gridmet', 
        'spei5y':'spei5y_gridmet',
        'pdsi':'pdsi_gridmet', 'pz':'pz_gridmet'
    }, inplace=True
)

df = pd.merge(dfx4_f, peakgridmet_extract, on = ['Plant_Biomass', 'Plot', 'Zone', 'Site', 'Year'], how='right',
              suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

display(df)

,Date_x,Latitude_x,Longitude_x,Plant_Biomass,Plot,QA_PIXEL,QA_RADSAT,SR_ATMOS_OPACITY,SR_B1,SR_B2,...,spei1y_Min,spei30d_Min,spei5y_Min,spei90d_Min,pdsi_gridmet,spei180d_gridmet,spei1y_gridmet,spei30d_gridmet,spei5y_gridmet,spei90d_gridmet
0,10/11/2000,31.538627,-81.422539,516.324,1,5440.000000,0.0,367.300000,0.044076,0.055722,...,-0.39,1.69,0.71,0.54,-2.930526,-0.747368,-1.363684,0.012105,-0.086316,-0.223684
1,NaN,NaN,NaN,1152.400,1,NaN,NaN,NaN,NaN,NaN,...,-0.32,1.45,0.47,-0.03,-3.202105,-0.996842,-1.439474,-0.214737,-0.070526,-0.615263
2,10/18/2000,31.518920,-81.228956,710.396,1,5474.133333,0.0,289.933333,0.048560,0.066871,...,-0.32,1.13,0.39,-0.03,-2.800526,-1.044211,-1.275263,-0.345789,-0.231053,-0.692632
3,10/10/2000,31.451676,-81.365498,1534.304,1,5463.272727,0.0,238.545455,0.039605,0.052045,...,-0.24,1.28,0.39,-0.17,-2.980000,-1.092632,-1.424737,-0.352632,-0.147368,-0.776316
4,10/10/2000,31.436090,-81.339919,2022.264,1,5440.000000,0.0,278.615385,0.043557,0.066665,...,-0.24,1.28,0.39,-0.17,-2.980000,-1.092632,-1.424737,-0.352632,-0.147368,-0.776316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3211,10/8/2019,31.334597,-81.478039,2365.620,8,21824.000000,0.0,NaN,0.012575,0.020196,...,1.45,1.01,1.45,0.32,1.553333,-0.579444,0.342778,-0.176111,1.287143,-0.306111
3212,10/8/2019,31.335017,-81.478711,1821.010,12,21824.000000,0.0,NaN,0.014710,0.021550,...,1.45,1.01,1.45,0.32,1.553333,-0.579444,0.342778,-0.176111,1.287143,-0.306111
3213,10/8/2019,31.334934,-81.478709,2482.584,13,21824.000000,0.0,NaN,0.014768,0.021763,...,1.45,1.01,1.45,0.32,1.553333,-0.579444,0.342778,-0.176111,1.287143,-0.306111
3214,10/8/2019,31.334682,-81.478564,2511.936,16,21824.000000,0.0,NaN,0.014768,0.021763,...,1.45,1.01,1.45,0.32,1.553333,-0.579444,0.342778,-0.176111,1.287143,-0.306111


In [31]:
for col in daymet_extract.columns:
    print(col)
    
##EXPORT
out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'datmetgridmet.csv')
# df.to_csv(out_csv, index = False)

Date
Latitude
Longitude
Plant_Biomass
Plot
Quadrat_Area
Sample_Size
Site
Species_Code
Year
Zone
dayl
index
prcp
srad
swe
tmax
tmin
vp


In [45]:
##Bands and indices
df['Sensor'] = np.where(df['Year']<2013, 'Landsat 5', 'Landsat 8') ##make sure no other sensors are being used

df.loc[df['Year'] == 2012, 'Sensor'] = 'Landsat 7'

df['ndvi'] = np.where(df['Sensor'] == 'Landsat 8', (df['SR_B5']-df['SR_B4'])/(df['SR_B5']+df['SR_B4']), \
                      (df['SR_B4']-df['SR_B3'])/(df['SR_B4']+df['SR_B3'])) 
##ndvi conditional based on whether sensor is Landsat-5 or 8

df['Blue_band'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B2'], df['SR_B1'])
df['Green_band'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B3'], df['SR_B2'])
df['Red_band'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B4'], df['SR_B3'])
df['NIR_band'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B5'], df['SR_B4'])
df['SWIR1_band'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B6'], df['SR_B5'])
df['SWIR2_band'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B7'], df['SR_B7'])

##Variables from Byrd et al. 2018 (make sure calculations are accurate):
df['savi'] = ((df['NIR_band']-df['Red_band'])*1.5)/(df['NIR_band']+df['Red_band']+0.5)
df['wdrvi5'] = (0.5*df['NIR_band']-df['Red_band'])/(0.5*df['NIR_band']+df['Red_band'])
df['nd_r_g'] = (df['Red_band']-df['Green_band'])/(df['Red_band']+df['Green_band'])
df['nd_g_b'] = (df['Green_band']-df['Blue_band'])/(df['Green_band']+df['Blue_band'])
df['nd_swir2_nir'] = (df['SWIR2_band']-df['NIR_band'])/(df['SWIR2_band']+df['NIR_band'])
df['nd_swir2_r'] = (df['SWIR2_band']-df['Red_band'])/(df['SWIR2_band']+df['Red_band'])

display(df)

##EXPORT
out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'df_mayoct.csv')
# df.to_csv(out_csv, index = False)

,Date_x,Latitude_x,Longitude_x,Plant_Biomass,Plot,QA_PIXEL,QA_RADSAT,SR_ATMOS_OPACITY,SR_B1,SR_B2,...,Red_band,NIR_band,SWIR1_band,SWIR2_band,savi,wdrvi5,nd_r_g,nd_g_b,nd_swir2_nir,nd_swir2_r
0,10/11/2000,31.538627,-81.422539,516.324,1,5440.000000,0.0,367.300000,0.044076,0.055722,...,0.053861,0.106292,0.083794,0.051344,0.119135,-0.006678,-0.016989,0.116697,-0.348572,-0.023918
1,NaN,NaN,NaN,1152.400,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10/18/2000,31.518920,-81.228956,710.396,1,5474.133333,0.0,289.933333,0.048560,0.066871,...,0.059497,0.110754,0.071124,0.038586,0.114710,-0.035870,-0.058351,0.158635,-0.483242,-0.213196
3,10/10/2000,31.451676,-81.365498,1534.304,1,5463.272727,0.0,238.545455,0.039605,0.052045,...,0.047582,0.081575,0.055488,0.031477,0.081043,-0.076893,-0.044792,0.135734,-0.443135,-0.203706
4,10/10/2000,31.436090,-81.339919,2022.264,1,5440.000000,0.0,278.615385,0.043557,0.066665,...,0.058874,0.151791,0.111497,0.063016,0.196118,0.126296,-0.062060,0.209653,-0.413275,0.033981
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3211,10/8/2019,31.334597,-81.478039,2365.620,8,21824.000000,0.0,NaN,0.012575,0.020196,...,0.039491,0.159492,0.078031,0.036925,0.257519,0.337601,-0.082100,0.394902,-0.624017,-0.033588
3212,10/8/2019,31.335017,-81.478711,1821.010,12,21824.000000,0.0,NaN,0.014710,0.021550,...,0.040267,0.208028,0.110234,0.052618,0.336285,0.441820,-0.087693,0.380374,-0.596246,0.132969
3213,10/8/2019,31.334934,-81.478709,2482.584,13,21824.000000,0.0,NaN,0.014768,0.021763,...,0.039243,0.166458,0.086199,0.042120,0.270401,0.359150,-0.080947,0.359139,-0.596120,0.035362
3214,10/8/2019,31.334682,-81.478564,2511.936,16,21824.000000,0.0,NaN,0.014768,0.021763,...,0.039243,0.166458,0.086199,0.042120,0.270401,0.359150,-0.080947,0.359139,-0.596120,0.035362


In [46]:
##THIS DOESNT HOLD UP W/ L9 DATA!!!! NEED TO ADJUST


df['ndvi_year'] = np.where(df['Sensor'] == 'Landsat 8', (df['SR_B5_year']-df['SR_B4_year'])/(df['SR_B5_year']+df['SR_B4_year']), \
                      (df['SR_B4_year']-df['SR_B3_year'])/(df['SR_B4_year']+df['SR_B3_year'])) 
##ndvi conditional based on whether sensor is Landsat-5 or 8 

df['Blue_band_year'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B2_year'], df['SR_B1_year'])
df['Green_band_year'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B3_year'], df['SR_B2_year'])
df['Red_band_year'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B4_year'], df['SR_B3_year'])
df['NIR_band_year'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B5_year'], df['SR_B4_year'])
df['SWIR1_band_year'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B6_year'], df['SR_B5_year'])
df['SWIR2_band_year'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B7_year'], df['SR_B7_year'])

##Variables from Byrd et al. 2018 (make sure calculations are accurate):
df['savi_year'] = ((df['NIR_band_year']-df['Red_band_year'])*1.5)/(df['NIR_band_year']+df['Red_band_year']+0.5)
df['wdrvi5_year'] = (0.5*df['NIR_band_year']-df['Red_band_year'])/(0.5*df['NIR_band_year']+df['Red_band_year'])
df['nd_r_g_year'] = (df['Red_band_year']-df['Green_band_year'])/(df['Red_band_year']+df['Green_band_year'])
df['nd_g_b_year'] = (df['Green_band_year']-df['Blue_band_year'])/(df['Green_band_year']+df['Blue_band_year'])
df['nd_swir2_nir_year'] = (df['SWIR2_band_year']-df['NIR_band_year'])/(df['SWIR2_band_year']+df['NIR_band_year'])
df['nd_swir2_r_year'] = (df['SWIR2_band_year']-df['Red_band_year'])/(df['SWIR2_band_year']+df['Red_band_year'])

In [47]:
##THIS DOESNT HOLD UP W/ L9 DATA!!!! NEED TO ADJUST

df['ndvi_peak'] = np.where(df['Sensor'] == 'Landsat 8', (df['SR_B5_peak']-df['SR_B4_peak'])/(df['SR_B5_peak']+df['SR_B4_peak']), \
                      (df['SR_B4_peak']-df['SR_B3_peak'])/(df['SR_B4_peak']+df['SR_B3_peak'])) 
##ndvi conditional based on whether sensor is Landsat-5 or 8

df['Blue_band_peak'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B2_peak'], df['SR_B1_peak'])
df['Green_band_peak'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B3_peak'], df['SR_B2_peak'])
df['Red_band_peak'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B4_peak'], df['SR_B3_peak'])
df['NIR_band_peak'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B5_peak'], df['SR_B4_peak'])
df['SWIR1_band_peak'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B6_peak'], df['SR_B5_peak'])
df['SWIR2_band_peak'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B7_peak'], df['SR_B7_peak'])

##Variables from Byrd et al. 2018 (make sure calculations are accurate):
df['savi_peak'] = ((df['NIR_band_peak']-df['Red_band_peak'])*1.5)/(df['NIR_band_peak']+df['Red_band_peak']+0.5)
df['wdrvi5_peak'] = (0.5*df['NIR_band_peak']-df['Red_band_peak'])/(0.5*df['NIR_band_peak']+df['Red_band_peak'])
df['nd_r_g_peak'] = (df['Red_band_peak']-df['Green_band_peak'])/(df['Red_band_peak']+df['Green_band_peak'])
df['nd_g_b_peak'] = (df['Green_band_peak']-df['Blue_band_peak'])/(df['Green_band_peak']+df['Blue_band_peak'])
df['nd_swir2_nir_peak'] = (df['SWIR2_band_peak']-df['NIR_band_peak'])/(df['SWIR2_band_peak']+df['NIR_band_peak'])
df['nd_swir2_r_peak'] = (df['SWIR2_band_peak']-df['Red_band_peak'])/(df['SWIR2_band_peak']+df['Red_band_peak'])

In [48]:
##THIS DOESNT HOLD UP W/ L9 DATA!!!! NEED TO ADJUST

df['ndvi_MarMay'] = np.where(df['Sensor'] == 'Landsat 8', (df['SR_B5_MarMay']-df['SR_B4_MarMay'])/(df['SR_B5_MarMay']+df['SR_B4_MarMay']), \
                      (df['SR_B4_MarMay']-df['SR_B3_MarMay'])/(df['SR_B4_MarMay']+df['SR_B3_MarMay'])) 
##ndvi conditional based on whether sensor is Landsat-5 or 8

df['Blue_band_MarMay'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B2_MarMay'], df['SR_B1_MarMay'])
df['Green_band_MarMay'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B3_MarMay'], df['SR_B2_MarMay'])
df['Red_band_MarMay'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B4_MarMay'], df['SR_B3_MarMay'])
df['NIR_band_MarMay'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B5_MarMay'], df['SR_B4_MarMay'])
df['SWIR1_band_MarMay'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B6_MarMay'], df['SR_B5_MarMay'])
df['SWIR2_band_MarMay'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B7_MarMay'], df['SR_B7_MarMay'])

##Variables from Byrd et al. 2018 (make sure calculations are accurate):
df['savi_MarMay'] = ((df['NIR_band_MarMay']-df['Red_band_MarMay'])*1.5)/(df['NIR_band_MarMay']+df['Red_band_MarMay']+0.5)
df['wdrvi5_MarMay'] = (0.5*df['NIR_band_MarMay']-df['Red_band_MarMay'])/(0.5*df['NIR_band_MarMay']+df['Red_band_MarMay'])
df['nd_r_g_MarMay'] = (df['Red_band_MarMay']-df['Green_band_MarMay'])/(df['Red_band_MarMay']+df['Green_band_MarMay'])
df['nd_g_b_MarMay'] = (df['Green_band_MarMay']-df['Blue_band_MarMay'])/(df['Green_band_MarMay']+df['Blue_band_MarMay'])
df['nd_swir2_nir_MarMay'] = (df['SWIR2_band_MarMay']-df['NIR_band_MarMay'])/(df['SWIR2_band_MarMay']+df['NIR_band_MarMay'])
df['nd_swir2_r_MarMay'] = (df['SWIR2_band_MarMay']-df['Red_band_MarMay'])/(df['SWIR2_band_MarMay']+df['Red_band_MarMay'])

In [49]:
##https://stackoverflow.com/questions/18557860/how-to-create-a-list-with-a-range-of-years

##WITH VARIABLE YEAR IN FILE NAME

df = df[df['Red_band'].notna()]
df = df[df['SR_B4_peak'].notna()]
df = df[df['SR_B4_year'].notna()]
df

,Date_x,Latitude_x,Longitude_x,Plant_Biomass,Plot,QA_PIXEL,QA_RADSAT,SR_ATMOS_OPACITY,SR_B1,SR_B2,...,Red_band_MarMay,NIR_band_MarMay,SWIR1_band_MarMay,SWIR2_band_MarMay,savi_MarMay,wdrvi5_MarMay,nd_r_g_MarMay,nd_g_b_MarMay,nd_swir2_nir_MarMay,nd_swir2_r_MarMay
0,10/11/2000,31.538627,-81.422539,516.324,1,5440.000000,0.0,367.300000,0.044076,0.055722,...,0.074358,0.139671,0.124326,0.072928,0.137206,-0.031367,0.008642,0.125736,-0.313936,-0.009709
2,10/18/2000,31.518920,-81.228956,710.396,1,5474.133333,0.0,289.933333,0.048560,0.066871,...,0.067042,0.124643,0.091324,0.053313,0.124916,-0.036486,-0.047063,0.141720,-0.400825,-0.114063
3,10/10/2000,31.451676,-81.365498,1534.304,1,5463.272727,0.0,238.545455,0.039605,0.052045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10/10/2000,31.436090,-81.339919,2022.264,1,5440.000000,0.0,278.615385,0.043557,0.066665,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,10/12/2000,31.388423,-81.279792,1032.632,1,5440.000000,0.0,309.857143,0.053210,0.071221,...,0.062383,0.119121,0.100778,0.060359,0.124881,-0.023146,-0.039911,0.142945,-0.327401,-0.016490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3211,10/8/2019,31.334597,-81.478039,2365.620,8,21824.000000,0.0,NaN,0.012575,0.020196,...,0.057559,0.118389,0.088827,0.051867,0.134988,0.014006,0.068127,0.265844,-0.390719,-0.052021
3212,10/8/2019,31.335017,-81.478711,1821.010,12,21824.000000,0.0,NaN,0.014710,0.021550,...,0.053528,0.149349,0.108869,0.061498,0.204490,0.164946,0.036641,0.276756,-0.416661,0.069285
3213,10/8/2019,31.334934,-81.478709,2482.584,13,21824.000000,0.0,NaN,0.014768,0.021763,...,0.056487,0.127410,0.097952,0.057746,0.155555,0.060052,0.059580,0.255843,-0.376239,0.011026
3214,10/8/2019,31.334682,-81.478564,2511.936,16,21824.000000,0.0,NaN,0.014768,0.021763,...,0.056487,0.127410,0.097952,0.057746,0.155555,0.060052,0.059580,0.255843,-0.376239,0.011026


In [50]:
#EXPORT
out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'ga_testx.csv')
# df.to_csv(out_csv, index = False)

## EXPORT
out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'df_testx.csv')
# df.to_csv(out_csv, index = False)